This tutorial is automatically generated from the file test/python/cell_based/tutorials/TestTensileTestTutorial.py.



In [ ]:
# Jupyter notebook specific imports 
import matplotlib as mpl 
import matplotlib.pyplot as plt 
%matplotlib inline


# Introduction
In this tutorial we will demonstrate a simulated tensile test on an epithelial sheet. This test
demonstrates:
 * Working with vertex based off lattice populations
 * Applying boundary conditions
 * Working with forces
 
## The Test



In [ ]:
import numpy as np
import chaste.core
chaste.init()
import chaste.cell_based
import chaste.mesh
import chaste.visualization

## Test 1 - A 2d test



First, we generate a vertex mesh using a HoneycombVertexMeshGenerator.



In [ ]:
generator = chaste.mesh.HoneycombVertexMeshGenerator(5, 15)
mesh = generator.GetMesh()

Now set up the cells, again we want to avoid proliferation.



In [ ]:
cells = chaste.cell_based.VecCellPtr()
differentiated_type = chaste.cell_based.DifferentiatedCellProliferativeType()
cell_generator = chaste.cell_based.CellsGeneratorUniformG1GenerationalCellCycleModel_2()
cell_generator.GenerateBasicRandom(cells, mesh.GetNumElements(), differentiated_type)

Next, create the cell population



In [ ]:
cell_population = chaste.cell_based.VertexBasedCellPopulation2(mesh, cells)

Pass the cell population into an `OffLatticeSimulation`, and set the output directory, output multiple and end time



In [ ]:
simulator = chaste.cell_based.OffLatticeSimulation2_2(cell_population)
simulator.SetOutputDirectory("Python/TestTensileTest")
simulator.SetEndTime(2.0)
simulator.SetSamplingTimestepMultiple(50)

Now create a force law



In [ ]:
force = chaste.cell_based.NagaiHondaForce2()
simulator.AddForce(force)

A NagaiHondaForce assumes that each cell has a target area. The target areas of cells are used to determine
pressure forces on each vertex and eventually determine the size of each cell in the simulation.
In order to assign target areas to cells and update them in each time step we add a SimpleTargetAreaModifier
to the simulation, which inherits from AbstractTargetAreaModifier.



In [ ]:
growth_modifier = chaste.cell_based.SimpleTargetAreaModifier2()
simulator.AddSimulationModifier(growth_modifier)

For our tensile test we will fix the bottom of the sheet and subject the top to an applied displacement. We neglect
fixing lateral degress of freedom for simplicity, since we are using an over-damped mechanical model.



In [ ]:
point = (0.0, 0.0)
normal = (0.0, -1.0)
bc = chaste.cell_based.AttractingPlaneBoundaryCondition2_2(cell_population, point, normal)
simulator.AddCellPopulationBoundaryCondition(bc)
normal = (0.0, 1.0)
bc2 = chaste.cell_based.AttractingPlaneBoundaryCondition2_2(cell_population, point, normal)
simulator.AddCellPopulationBoundaryCondition(bc2)

We want to displace our top boundary over time. We could write a custom boundary condition class to do this.
A more simple alternative is to modify the the position of the point describing our boundary plane in `bc2`
as the simulation progresses. As per earlier tutorials we make a new `SimulationModifier` class to do this.



In [ ]:
    """ Class for time varying boundary conditions
    """
        self.boundary_condition = boundary_condition
        self.original_location = boundary_condition.rGetPointOnPlane()
        self.velocity = 0.5 # cell lengths per time
        super(BoundaryConditionModifier, self).__init__()
        """ Move the boundary upwards at the specified velocity
        """
        total_time = chaste.cell_based.SimulationTime.Instance().GetTime()
        new_location = [self.original_location[0], self.original_location[1]+self.velocity*total_time]
        self.boundary_condition.SetPointOnPlane(np.array(new_location))
        """ Make sure the cell population is in the correct state at the start of the simulation
        """
        cell_population.Update()
        """ This needs to be explicitly over-ridden as the C++ method is pure virtual.
        """
        pass
bc_modifier = BoundaryConditionModifier(bc2)
simulator.AddSimulationModifier(bc_modifier)

PyChaste can do simple 3D rendering with VTK. We set up a `VtkScene` so that we can see the population
evovle in real time.



In [ ]:
scene= chaste.visualization.VtkScene2()
scene.SetIsInteractive(True)
scene.SetCellPopulation(cell_population)

Save snapshot images of the population during the simulation



In [ ]:
scene_modifier = chaste.cell_based.VtkSceneModifier2()
scene_modifier.SetVtkScene(scene)
scene_modifier.SetUpdateFrequency(100)
simulator.AddSimulationModifier(scene_modifier)

To run the simulation, we call `Solve()`.



In [ ]:
scene.Start()
simulator.Solve();
scene.StartInteractiveEventHandler()